In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
import re
import math
import os

In [2]:
driver = webdriver.Chrome(r"C:\Users\Donovan\Downloads\chromedriver_win32\chromedriver.exe")

In [3]:
def getChannels(url):
    driver.get(url)
    content = driver.page_source
    soup = BeautifulSoup(content)
    channels = []

    num_pages = math.ceil(int(soup.find('p',{'class':'pagination'}).text.split(" ")[0][1:].replace(",","")) / 10)
        
    for page in range(1, num_pages+1):
        driver.get(url+"?&page="+str(page))
        content = driver.page_source
        soup = BeautifulSoup(content)
        for _ in soup.findAll('a',href=re.compile("\/discord\/channel\/.+")):
            channels.append(_['href'])
    
    return channels

In [4]:
url = "https://discordleaks.unicornriot.ninja"
channels = getChannels(url+'/discord/server/')

In [5]:
def createCSV(url):
    username=[] #Discord Username
    user_id=[] #User ID from Unicorn Database (not the same as the Discord ID)
    message_id=[] #Message ID from the Unicorn Database
    avatar=[] #Avatar image link
    timestamp=[] #Time of message posting
    message=[] #Message content
    
    driver.get(url)
    content = driver.page_source
    soup = BeautifulSoup(content)
    
    num_pages = math.ceil(int(soup.find('p',{'class':'pagination'}).text.split(" ")[0][1:].replace(",","")) / 250)
    
    for page in range(1,num_pages+1):
        driver.get(url+"?&page="+str(page))
        content = driver.page_source
        soup = BeautifulSoup(content)
        
        chatroom = []
        for _ in soup.findAll('div',{'class' : 'discord-message'}):
            chatroom.append(_)

        for _ in range(len(chatroom)):
            message_id.append(chatroom[_].find('a', href=re.compile("\/discord\/view\/.+"))['href'][14:-7])
            user_id.append(chatroom[_].find('a', href=re.compile("\/discord\/user\/.+"))['href'][14:])
            avatar.append(chatroom[_].find('a', href=re.compile("\/discord\/user\/.+")).find('img')['src'])
            username.append(chatroom[_].find('div', {'class':'discord-message-user-name'}).find('a').text[1:-1])
            timestamp.append(chatroom[_].find('div', {'class':'discord-message-meta-items'}).find('span').text)
            message.append(chatroom[_].find('div', {'class':'discord-message-content'}).text)
    
    df = pd.DataFrame({'Timestamp':timestamp,'User ID':user_id,'Username':username,'Avatar':avatar,'Message ID':message_id,'Message Content':message})
    outname = soup.title.text.split(" ")[0][1:]+".csv"
    outdir = "./"+re.sub(r'[#\/?!,]','',"_".join(soup.title.text[soup.title.text.find("in\n"):soup.title.text.find(" | ")].split(" ")[3:-2]))
    if not os.path.exists(outdir):
        os.mkdir(outdir)
    fullname = os.path.join(outdir, outname)    
    df.to_csv(fullname, index=False, encoding='utf-8')

In [6]:
for _ in channels[50:100]:
    current_url = url + _
    createCSV(current_url)